# Pull in dropped ID back to y_pred , y_test

id = df['id']

y=df['target']
X=df.drop('target', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size =0.8)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

results = y_test.to_frame('Actual Value')
results.insert(0, 'Predicted Value'), y_pred)
results = id.to_frame().join(results, how='inner')

# Drop Columns Function for Pipeline

In [ ]:
#remove columns
from sklearn.preprocessing import FunctionTransformer

def drop_cols(df,cols_to_drop):

    df_copy = df.copy()
    
    ret_df = df_copy.drop(columns=cols_to_drop)
    return ret_df

cols_to_drop = ['trans_date_trans_time', 'merchant', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 
               'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'hour', 'weekday', 'region', 'age']
drop_cols_trans = FunctionTransformer(drop_cols, kw_args={'cols_to_drop':cols_to_drop})


# chi square 

In [ ]:
from scipy import stats

crosstab = pd.crosstab(df['gender'], df['is_fraud'])
print(crosstab)
stats.chi2_contingency(crosstab)

# correlation heatmap

In [ ]:
import matplotlib.pyplot as plt
from datetime import timedelta
df['dob'] = pd.to_datetime(df['dob'], format='%Y-%m-%d')
df['age'] = (date.today() - df['dob']) // timedelta(days=365.2425)

corr = df.corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

# describe for categorical fields

In [ ]:
print(app_record.describe(include = 'object'))

# get feature names from preprocessing pipeline

In [ ]:
import warnings
import sklearn

def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names


get_feature_names(preprocessor)